# Feature extraction and EDA 

## 1. Import libraries and explore data

In [1]:
#import libraries
import pandas as pd
import numpy as np
import collections
import re
import sys
import time
import matplotlib.pyplot as plt
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import wordninja
from itertools import islice
import pickle
from sklearn.model_selection import train_test_split

In [2]:
df = pd.DataFrame(pd.read_pickle('df_music'))

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5100 entries, 0 to 5099
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Rank              5100 non-null   int64  
 1   Song              5100 non-null   object 
 2   Artist            5100 non-null   object 
 3   Year              5100 non-null   int64  
 4   Lyrics            4913 non-null   object 
 5   Source            4913 non-null   float64
 6   Artists clean     5100 non-null   object 
 7   artist_song1      5100 non-null   object 
 8   songs_clean       5100 non-null   object 
 9   artist_song2      5100 non-null   object 
 10  danceability      5083 non-null   object 
 11  energy            5083 non-null   object 
 12  key               5083 non-null   object 
 13  loudness          5083 non-null   object 
 14  mode              5083 non-null   object 
 15  speechiness       5083 non-null   object 
 16  acousticness      5083 non-null   object 


In [4]:
df.shape

(5100, 29)

## 2. Drop songs without or with missing lyrics

In [5]:
# drop rows with missing values

df = df[df.Lyrics != " NA "]
df.dropna( how='any', subset=['Lyrics'], inplace=True)
#reset index
df.reset_index(inplace=True, drop=True)
print("The number of remaining songs is: ", df.shape[0])

The number of remaining songs is:  4897


In [6]:
#remove songs that are instrumental

df=df[df['Lyrics']!='instrumental'] 
df=df[df['Lyrics']!=' instrumental'] 
df=df[df['Lyrics']!=' instrumental '] 
df=df[df['Lyrics']!='instrumental ']
print("The number of remaining songs is: ", df.shape[0])

The number of remaining songs is:  4878


## 3. Clean lyrics and tokenize

In [7]:
# clean lyrics - remove punctuation


In [8]:
lyrics=[]

for lyric in df.Lyrics: 
    lyric_string=re.sub('[^A-Za-z]+', ' ', lyric)
    lyrics_string = re.sub('/\s\s+/g', ' ', lyric)
    lyrics.append(lyric_string.lstrip())


In [9]:
# apply wordninja as some lyrics have words crunched together in one string
lyrics_clean=[]
for lyric in lyrics :
    string=wordninja.split(lyric)
    title=""
    for s in string:
        title+=s+" " 
    lyrics_clean.append(title.strip())

In [10]:
#add to df
df['lyrics_clean']=lyrics_clean

In [11]:
#apply lemmatizer and tokenizer
# apply word tokenizer, delete stopwords, and apply lemmatizer
tokens=[]
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

for row in lyrics_clean:
    row_tokens=word_tokenize(row)
    filtered_sent = [w for w in row_tokens if not w.lower() in stop_words]
    stemmed = [lemmatizer.lemmatize(word) for word in filtered_sent]
    tokens.append(stemmed)
df['tokens']=tokens

In [12]:
df[['tokens', 'Rank']].to_pickle('df_tokenized')

## 4. Make more features

In [13]:
import warnings
warnings.filterwarnings('ignore')
# Word frequency features
df.unique_words=df.tokens.apply(lambda x: len(set(x) ))
df.total_words=df.tokens.apply(lambda x: len(x) )
df.word_length=df.tokens.apply(lambda x: np.mean([len(word) for word in x]))

#keep only rows where no of words !=0
df=df[df.unique_words!=0]
df.shape

(4831, 31)

In [14]:
# get types of words per song by POS tagging
from nltk.tag import pos_tag
import nltk 
#nltk.download('tagsets')


In [15]:
#list of pos tags per text
def get_pos(text):
    pos=[]
    for i in range(len(text)):
        pos.append(pos_tag(text)[i][1])
    return dict(pd.Series(pos).value_counts())
pos_count=list(df.tokens.apply(get_pos))

In [16]:
#generate features for word difficulty 

import syllables
import spacy
from textstat.textstat import textstatistics, legacy_round

def syllables_count(word):
    return syllables.estimate(word)
 
# Returns the average number of syllables per word in the text
def avg_syllables_per_word(text):
    syllable = syllables_count(text)
    words = word_count(text)
    ASPW = float(syllable) / float(words)
    return legacy_round(ASPW, 1)

# Return total no of Difficult Words in a text (difficult words are those with syllables >= 2)
    # a list of common words)
def difficult_words(text):
    diff_words_set = set()  
    nlp = spacy.load('en_core_web_sm')
    for word in text:
        syllable_count = syllables_count(word)
        if word not in nlp.Defaults.stop_words and syllable_count >= 2:
            diff_words_set.add(word)
    return len(diff_words_set)
 
# Count of  polysyllablic words (if word has more than 3 syllables)

def poly_syllable_count(text):
    count = 0  
    for word in text:
        syllable_count = syllables_count(word)
        if syllable_count >= 3:
            count += 1
    return count

In [17]:
# find number of words per song that are perfect rhymes 
#perfect rhymes: DOG -> COG

from Phyme import Phyme

ph = Phyme()
def get_rhymes(word):
    list_dict=[value for key,value in ph.get_perfect_rhymes(word).items()]
    all_rhymes=[]
    for row in list_dict:
        for word in row:
            all_rhymes.append(word)
    all_rhymes=list(set(all_rhymes))
    return all_rhymes
    
def count_rhymes(text):
    rhyme_count=0
    for word in list(set(text)):
        try:
            all_rhymes=get_rhymes(word) # get list of all words that rhyme
            for x in all_rhymes: # for each of these words, check if it is contained in the lyrics
                if x in text:
                    rhyme_count+=1
                else: 
                    pass
        except:
            pass
    return rhyme_count        

In [18]:
def add_text_features(df):
    df.polysyl_count=df.tokens.apply(poly_syllable_count)
    df.difficult_count=df.tokens.apply(difficult_words)
    df.count_rhyming_words=df.tokens.apply(count_rhymes)
    return df

In [19]:
#parallelize - it seems to take even longer??
import random
from multiprocessing import  Pool
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df


In [22]:
df.to_pickle("df_add_feat") # save df

In [ ]:
#PARALLELIZE - it seems like this takes longer?? HOW
addfeat = parallelize_dataframe(df, add_text_features)

In [ ]:
#addfeat = add_features(df_test) # takes about a day to run

In [ ]:
df_count=pd.DataFrame(pos_count)
df=addfeat.join(df_count, how='inner')

In [ ]:
#make plot of no of difficult words by chart ranking
import seaborn as sns
sns.set_theme(style="whitegrid")
df_grouped=df["Rank", "difficult_count"].groupby(by=["Rank"]).mean()
ax = sns.barplot(x="Rank", y="difficult_count", data=df)


In [ ]:
df.to_pickle("df_add_feat") # save df